# 3. Solutions to GTFS Reconstruction

This notebook demonstrates converting optimization solutions back to GTFS feeds. It covers:

3.1 Quick Problem Setup
- Load GTFS data and create optimization problem
- Run a simple optimization to get solutions

3.2 Phase 1: Solution Conversion
- Convert solution matrices to headway values
- Validate solutions and extract templates

3.3 Phase 2: GTFS Reconstruction
- Generate new trips and stop times
- Create complete GTFS feed

3.4 Validation and Analysis
- Compare original vs reconstructed GTFS
- Verify service patterns match optimization

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


print("=== GTFS RECONSTRUCTION WORKFLOW ===")
print("🔄 Testing solution-to-GTFS conversion pipeline")
print(f"📁 Project root: {project_root}")

=== GTFS RECONSTRUCTION WORKFLOW ===
🔄 Testing solution-to-GTFS conversion pipeline
📁 Project root: /home/hussein/Documents/GitHub/transit_opt


## 3.1 Quick Problem Setup

Create a minimal optimization problem to generate test solutions.

In [2]:
from transit_opt.preprocessing.prepare_gtfs import GTFSDataPreparator
from transit_opt.optimisation.config.config_manager import OptimizationConfigManager
from transit_opt.optimisation.runners.pso_runner import PSORunner

print("=== LOADING GTFS DATA ===")

# Quick GTFS setup
preparator = GTFSDataPreparator(
    gtfs_path='../data/external/study_area_gtfs_bus.zip',
    interval_hours=6,  # 4 periods per day for simplicity
    date=None,
    turnaround_buffer=1.15,
    max_round_trip_minutes=240.0,
    no_service_threshold_minutes=480.0,
    log_level="INFO"
)

# Simple headway choices for testing
allowed_headways = [10, 15, 30, 60, 120]
opt_data = preparator.extract_optimization_data(allowed_headways)

print(f"✅ Loaded {opt_data['n_routes']} routes, {opt_data['n_intervals']} intervals")
print(f"📊 Headway choices: {allowed_headways}")
print(f"🎯 Decision variables: {np.prod(opt_data['decision_matrix_shape'])}")

2025-09-25 16:22:42,923 - transit_opt.preprocessing.prepare_gtfs - INFO - Initializing GTFSDataPreparator with 6h intervals
2025-09-25 16:22:42,923 - transit_opt.preprocessing.prepare_gtfs - INFO - Loading GTFS feed from ../data/external/study_area_gtfs_bus.zip


=== LOADING GTFS DATA ===


2025-09-25 16:22:45,317 - transit_opt.preprocessing.prepare_gtfs - INFO - Using full GTFS feed (all service periods)
2025-09-25 16:22:46,686 - transit_opt.preprocessing.prepare_gtfs - INFO - GTFS loaded and cached in 3.76 seconds
2025-09-25 16:22:46,686 - transit_opt.preprocessing.prepare_gtfs - INFO - Dataset: 13,974 trips, 703,721 stop times
2025-09-25 16:22:46,687 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting optimization data with 5 allowed headways
2025-09-25 16:22:46,687 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting route essentials with 6-hour intervals
2025-09-25 16:22:47,680 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 73302: Round-trip 317.4min exceeds limit (240.0min), filtered out
2025-09-25 16:22:47,715 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 54721: Round-trip 366.8min exceeds limit (240.0min), filtered out
2025-09-25 16:22:48,926 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 30922: Round-trip 247.

✅ Loaded 147 routes, 4 intervals
📊 Headway choices: [10, 15, 30, 60, 120]
🎯 Decision variables: 588


In [3]:
# Quick optimization to get test solutions
print("=== RUNNING QUICK OPTIMIZATION ===")

# Minimal config for fast testing
test_config = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 2.0,  # Larger zones for speed
            'crs': 'EPSG:3857',
            'boundary_file': '../data/external/boundaries/study_area_boundary.geojson',
            'boundary_buffer_km': 2.0
        },
        'constraints': [
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.3,  # Max 20% more than current
                'measure': 'peak'
            },
            {
                'type': 'MinimumFleetConstraintHandler',
                'min_fleet_fraction': 0.8,  # Maintain 80% of current service
                'level': 'system',
                'measure': 'peak', 
                'baseline': 'current_peak'
            }
        ]
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 25,  # Small for speed
            'inertia_weight': 0.9,
            'inertia_weight_final': 0.4,
            'cognitive_coeff': 2.0,
            'social_coeff': 2.0,
            'use_penalty_method': False
        },
        'termination': {
            'max_generations': 30  # Very short for testing
        },
        'monitoring': {
            'progress_frequency': 5,
            'save_history': True,
            'detailed_logging': False
        }
    }
}

# Run optimization
config_manager = OptimizationConfigManager(config_dict=test_config)
pso_runner = PSORunner(config_manager)
result = pso_runner.optimize(opt_data, track_best_n=5)

print(f"✅ Optimization complete: {result.best_objective:.4f}")
print(f"📊 Found {len(result.best_feasible_solutions) if hasattr(result, 'best_feasible_solutions') and result.best_feasible_solutions else 0} feasible solutions")
print(f"⏱️  Time: {result.optimization_time:.1f}s")

=== RUNNING QUICK OPTIMIZATION ===
📋 Using provided configuration dictionary
🚀 STARTING PSO OPTIMIZATION
🗺️ Setting up spatial analysis with 2.0km resolution
🗺️  Reprojected 6897 stops to EPSG:3857
🔧 Creating 219 × 387 = 84753 grid cells
   Grid bounds: (-452551, 6584519) to (-15585, 7357992) meters
   Cell size: 2000.0m × 2000.0m
✅ Created 84753 hexagonal zones in EPSG:3857
🚀 Using spatial join for zone mapping...
✅ Mapped 6897 stops to zones
🚀 Pre-computing route-stop mappings...
✅ Cached stops for 187 routes
✅ Spatial system ready: 84753 hexagonal zones
   📋 Creating 2 constraint handler(s)...
      Creating constraint 1: FleetTotalConstraintHandler
         ✓ FleetTotal: 1 constraint(s)
      Creating constraint 2: MinimumFleetConstraintHandler
         ✓ MinimumFleet: 1 constraint(s)
🏗️  CREATING TRANSIT OPTIMIZATION PROBLEM:
   📊 Problem dimensions:
      Routes: 147
      Time intervals: 4
      Headway choices: 6
   🔧 Pymoo parameters:
      Decision variables: 588
      Object

## 3.2 Phase 1: Solution Conversion

Test the SolutionConverter class for converting optimization results to headway specifications.

In [4]:
from transit_opt.gtfs.gtfs import SolutionConverter

print("=== TESTING SOLUTION CONVERTER ===")

# Create converter
converter = SolutionConverter(opt_data)

# Test with best solution
test_solution = result.best_solution
print(f"🔍 Testing solution shape: {test_solution.shape}")

# Validate solution
validation = converter.validate_solution(test_solution)
print(f"\n✅ VALIDATION RESULTS:")
print(f"   Valid: {validation['valid']}")
print(f"   Errors: {len(validation['errors'])}")
print(f"   Warnings: {len(validation['warnings'])}")

if validation['errors']:
    for error in validation['errors']:
        print(f"   ❌ {error}")

# Show statistics
stats = validation['statistics']
print(f"\n📊 SOLUTION STATISTICS:")
print(f"   Service coverage: {stats['service_percentage']:.1f}%")
print(f"   Active cells: {stats['service_cells']}/{stats['total_cells']}")
print(f"   Headway distribution: {stats['headway_distribution']}")

=== TESTING SOLUTION CONVERTER ===
🔍 Testing solution shape: (147, 4)

✅ VALIDATION RESULTS:
   Valid: True
   Errors: 0
   Warnings: 0

📊 SOLUTION STATISTICS:
   Service coverage: 90.8%
   Active cells: 534/588
   Headway distribution: {'10min': 61, '15min': 112, '30min': 140, '60min': 143, '120min': 78, 'No Service': 54}


In [5]:
# Convert to headways dictionary
print("=== CONVERTING TO HEADWAYS ===")

headways_dict = converter.solution_to_headways(test_solution)
print(f"📋 Converted {len(headways_dict)} routes")

# Show example route
example_route = list(headways_dict.keys())[25]
print(f"\n📋 EXAMPLE ROUTE ({example_route}):")
for interval, headway in headways_dict[example_route].items():
    if headway is None:
        print(f"   {interval:>15}: No Service")
    else:
        print(f"   {interval:>15}: {headway:.0f} minutes")

# Extract template trips
print(f"\n🔧 EXTRACTING TEMPLATE TRIPS:")
templates = converter.extract_route_templates()
print(f"   Templates extracted: {len(templates)}/{len(headways_dict)}")

if templates:
    example_template = templates[example_route]

    for interval_label, interval_data in example_template.items():
        print(f"Interval: {interval_label}")
        print(f"  Trip ID: {interval_data['trip_id']}")
        print(f"  Duration: {interval_data['duration_minutes']:.1f} min")
        print(f"  Stops: {interval_data['n_stops']}")
        print(f"  Route Info: {interval_data['route_info']}")
        print()

=== CONVERTING TO HEADWAYS ===
📋 Converted 147 routes

📋 EXAMPLE ROUTE (12594):
            00-06h: 60 minutes
            06-12h: 120 minutes
            12-18h: 15 minutes
            18-24h: No Service

🔧 EXTRACTING TEMPLATE TRIPS:
🔍 Processing route 50627...
   ✅ 00-06h: 34.0min template
   ✅ 06-12h: 43.0min template
   ✅ 12-18h: 43.0min template
   ✅ 18-24h: 35.0min template
🔍 Processing route 50628...
   ✅ 00-06h: 65.0min template
   ✅ 06-12h: 79.0min template
   ✅ 12-18h: 80.0min template
   ✅ 18-24h: 64.0min template
🔍 Processing route 50629...
   ✅ 00-06h: 72.0min template
   ✅ 06-12h: 72.0min template
   ✅ 12-18h: 74.0min template
   ✅ 18-24h: 63.0min template
🔍 Processing route 58940...
   ⚠️  00-06h: Using fallback template (30.0min)
   ✅ 06-12h: 20.0min template
   ✅ 12-18h: 30.0min template
   ⚠️  18-24h: Using fallback template (30.0min)
🔍 Processing route 22577...
   ✅ 00-06h: 74.0min template
   ✅ 06-12h: 84.0min template
   ✅ 12-18h: 84.0min template
   ✅ 18-24h: 71.0

## 3.3 Phase 3: GTFS Reconstruction 

This section will go over steps to 
 - extract the top N solutions
 - Update files for GTFS (trips.txt and stop_times.txt) based on headway vales in solution
 - GTFS file creation

### 3.3.1 Extract top N solutions

In [6]:
# Extract top N solutions from optimization results
print("=== PROCESSING TOP N SOLUTIONS ===")

# Get the best solutions (you can adjust N as needed)
N_SOLUTIONS = 5
top_solutions = []

if hasattr(result, 'best_feasible_solutions') and result.best_feasible_solutions:
    # Use feasible solutions if available
    top_solutions = result.best_feasible_solutions[:N_SOLUTIONS]
    print(f"📊 Using top {len(top_solutions)} feasible solutions")
else:
    # Fall back to best solution
    top_solutions = [result.best_solution]
    print(f"📊 Using best solution only (no feasible solutions tracked)")

print(f"✅ Will process {len(top_solutions)} solutions")



=== PROCESSING TOP N SOLUTIONS ===
📊 Using top 5 feasible solutions
✅ Will process 5 solutions


In [7]:
top_solutions

[{'solution': array([[4, 4, 4, 5],
         [3, 0, 1, 1],
         [2, 4, 0, 2],
         [3, 1, 3, 1],
         [2, 2, 0, 2],
         [3, 1, 2, 3],
         [5, 3, 3, 4],
         [5, 3, 1, 2],
         [3, 5, 3, 5],
         [5, 3, 3, 2],
         [4, 5, 1, 5],
         [2, 2, 2, 1],
         [2, 1, 2, 2],
         [2, 1, 4, 2],
         [2, 3, 0, 1],
         [3, 4, 5, 1],
         [5, 2, 1, 2],
         [4, 4, 1, 1],
         [2, 2, 4, 2],
         [4, 2, 1, 1],
         [2, 3, 2, 4],
         [1, 3, 1, 1],
         [1, 2, 5, 4],
         [3, 2, 0, 1],
         [5, 3, 2, 2],
         [3, 4, 1, 5],
         [3, 2, 2, 3],
         [4, 2, 2, 1],
         [3, 2, 5, 2],
         [3, 2, 0, 0],
         [3, 1, 3, 3],
         [4, 3, 2, 2],
         [4, 5, 0, 3],
         [2, 1, 2, 5],
         [0, 2, 1, 0],
         [2, 1, 0, 5],
         [3, 3, 4, 1],
         [5, 2, 4, 3],
         [3, 2, 3, 5],
         [1, 1, 4, 2],
         [3, 1, 1, 3],
         [2, 3, 0, 3],
         [4, 3, 2, 3],

### 3.3.2 Create GTFS Feed

In [11]:
print("=== GENERATING GTFS FEEDS FOR TOP SOLUTIONS ===")

# Create converter once
converter = SolutionConverter(opt_data)

generated_feeds = []

for i, sol in enumerate(top_solutions, 1):
    solution_matrix = sol['solution'] if isinstance(sol, dict) and 'solution' in sol else sol
    print(f"\n🔄 Processing Solution #{i} (shape: {getattr(solution_matrix, 'shape', 'No shape')})")
    
    # Step 1: Validate solution
    validation = converter.validate_solution(solution_matrix)
    if not validation['valid']:
        print(f"   ❌ Invalid solution: {validation['errors']}")
        continue
    
    # Show solution stats
    stats = validation['statistics']
    print(f"   📊 Service coverage: {stats['service_percentage']:.1f}%")
    print(f"   🚌 Active routes: {stats['service_cells']}/{stats['total_cells']} cells")
    
    # Step 2: Convert to headways
    headways_dict = converter.solution_to_headways(solution_matrix)
    
    # Step 3: Extract templates (done once, reused for all solutions)
    if i == 1:  # Only extract templates once
        templates = converter.extract_route_templates()
        print(f"   🔧 Extracted templates for {len(templates)} routes")
    
    # Step 4: Generate GTFS feed
    try:
        service_id = f'optimized_solution_{i:02d}'
        gtfs_path = converter.build_complete_gtfs(
            headways_dict, 
            templates,
            service_id=service_id,
            output_dir=f'output/solution_{i:02d}_gtfs',
            zip_output=True
        )
        
        generated_feeds.append({
            'solution_id': i,
            'path': gtfs_path,
            'service_coverage': stats['service_percentage'],
            'objective_value': sol['objective'] if isinstance(sol, dict) and 'objective' in sol else 'N/A'
        })
        
        print(f"   ✅ GTFS created: {gtfs_path}")
        
    except Exception as e:
        print(f"   ❌ GTFS generation failed: {e}")

# Summary of generated feeds
print(f"\n📁 GENERATED GTFS FEEDS SUMMARY:")
print(f"{'ID':<4} {'Coverage':<10} {'Objective':<12} {'Path'}")
print("-" * 60)

for feed in generated_feeds:
    coverage = f"{feed['service_coverage']:.1f}%"
    objective = f"{feed['objective_value']:.4f}" if feed['objective_value'] != 'N/A' else 'N/A'
    print(f"{feed['solution_id']:<4} {coverage:<10} {objective:<12} {feed['path']}")

print(f"\n✅ Successfully generated {len(generated_feeds)} GTFS feeds from top solutions!")

=== GENERATING GTFS FEEDS FOR TOP SOLUTIONS ===

🔄 Processing Solution #1 (shape: (147, 4))
   📊 Service coverage: 90.8%
   🚌 Active routes: 534/588 cells
🔍 Processing route 50627...
   ✅ 00-06h: 34.0min template
   ✅ 06-12h: 43.0min template
   ✅ 12-18h: 43.0min template
   ✅ 18-24h: 35.0min template
🔍 Processing route 50628...
   ✅ 00-06h: 65.0min template
   ✅ 06-12h: 79.0min template
   ✅ 12-18h: 80.0min template
   ✅ 18-24h: 64.0min template
🔍 Processing route 50629...
   ✅ 00-06h: 72.0min template
   ✅ 06-12h: 72.0min template
   ✅ 12-18h: 74.0min template
   ✅ 18-24h: 63.0min template
🔍 Processing route 58940...
   ⚠️  00-06h: Using fallback template (30.0min)
   ✅ 06-12h: 20.0min template
   ✅ 12-18h: 30.0min template
   ⚠️  18-24h: Using fallback template (30.0min)
🔍 Processing route 22577...
   ✅ 00-06h: 74.0min template
   ✅ 06-12h: 84.0min template
   ✅ 12-18h: 84.0min template
   ✅ 18-24h: 71.0min template
🔍 Processing route 29092...
   ⚠️  00-06h: Using fallback template 

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 6917 trips
✅ Generated stop_times.txt: 307710 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 6866 trips
✅ Generated stop_times.txt: 306701 stop times
📅 Using m

...

⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 6913 trips
✅ Generated stop_times.txt: 308327 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 6909 trips
✅ Generated stop_times.txt: 306868 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 6910 trips
✅ Generated stop_times.txt: 307757 stop times
📅 Using m